#### Homework Solution for Week 4

In [1]:
# !wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
# !unzip bank+marketing.zip 
# !unzip bank.zip

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
raw_data = pd.read_csv("/app/new/splatter-image/DTC/Machine Learning Zoomcamp/week3/bank-full.csv", sep=";")
raw_data.columns = raw_data.columns.str.lower().str.replace(' ', '_')
raw_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
used_columns = ["age", "job", "marital", "education", "balance", 
                "housing", "contact", "day", "month", "duration", 
                "campaign", "pdays", "previous", "poutcome", "y"]
sample_data = raw_data.loc[:, used_columns]
sample_data.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
categorical_columns = sample_data.columns[sample_data.dtypes == "object"].values[:-1]
numerical_columns = sample_data.columns[sample_data.dtypes != "object"].values
categorical_columns, numerical_columns

(array(['job', 'marital', 'education', 'housing', 'contact', 'month',
        'poutcome'], dtype=object),
 array(['age', 'balance', 'day', 'duration', 'campaign', 'pdays',
        'previous'], dtype=object))

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score

X = sample_data.iloc[:, :-1]
y = sample_data["y"].map({'yes': 1.0, 'no': 0.0})

X_full_train, X_test, y_full_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=1)

X_full_train.reset_index(drop=True, inplace=True)
X_train.reset_index(drop=True, inplace=True)
X_val.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

y_full_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_val.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

##### Loading the model and inferring on the dataset

In [7]:
def predict(X, dict_transformer, model):
    x_dict = dict_transformer.transform(X.to_dict(orient="records"))
    return model.predict_proba(x_dict)[:, 1]

In [8]:
# # # let's use pickle to save the model s.t. we can later access it
import pickle

C = 1.0
output_file = f'model_{str(C).replace(".", "_")}.bin'

with open(output_file, "rb") as inF:
    dict_transformer, model = pickle.load(inF)

In [9]:
test_score = roc_auc_score(y_true=y_test, y_score=predict(X_test, dict_transformer, model))
print(f"Model's generalization performance --> AUC = {np.round(test_score, decimals=3)}")

Model's generalization performance --> AUC = 0.906


##### Performing inference on the new customer

In [10]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

x_customer = dict_transformer.transform([customer])

In [13]:
model.predict_proba(x_customer)[0, 1]

0.2838710124958317